# Tensorflow MNIST-LeNet Patch Demo

This notebook demonstrates the adversarial patch attack applied on the LeNet model, as well as preprocessing and adversarial training defenses that can be run on any modern laptop.
Please see the [example README](README.md) for instructions on how to prepare your environment for running this example.

## Setup: Experiment Name and MNIST Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

In [ ]:
# Import packages from the Python standard library
import importlib.util
import sys
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = "mnist_adversarial_patch"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:80"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
if os.getenv("DIOPTRA_RESTAPI_URI") is None:
    os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

from scripts.client import DioptraClient
from scripts.utils import make_tar

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `src/MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, we provide the `make_tar` helper function defined in `examples/scripts/utils.py`.

In [ ]:
make_tar(["src"], WORKFLOWS_TAR_GZ)

To connect with the endpoint, we will use a client class defined in the `examples/scripts/client.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below.
The client uses the environment variable `DIOPTRA_RESTAPI_URI`, which we configured at the top of the notebook, to figure out how to connect to the Dioptra RESTful API.

In [ ]:
restapi_client = DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment named `"mnist_adversarial_patch"` exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

## Adversarial Patches: Baseline MNIST Training

Now, we will train our baseline LeNet-5 model on the MNIST dataset. 
We will be submitting our jobs to the `"tensorflow_gpu"` queue.
Once the experiment is finished, we will examine the accuracy results of our model.

In [ ]:
# Create and submit training job.
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_le_net",
        "-P model_architecture=le_net",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [ ]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])

    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)

    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])

    return job_response

Now wait for the job to complete before proceeding to next steps.

In [ ]:
response_le_net_train = wait_until_finished(response_le_net_train)
print("Training job for LeNet-5 neural network")
pprint.pprint(response_le_net_train)

### Checking baseline MNIST job accuracy

Once the job has finished running we can view the results either through the MLflow URI or by accessing the job via MLflow client.
Here we will show the baseline accuracy results from the previous training job.
Please see [Querying the MLFlow Tracking Service](#Querying-the-MLFlow-Tracking-Service) section for more details.

In [ ]:
# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    if(job_response['status']=="failed"):
        return {}

    run = mlflow_client.get_run(job_response["mlflowRunId"])
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])

    return run


results = get_mlflow_results(response_le_net_train)
pprint.pprint(results.data.metrics)

With the baseline training settings, it appears that the LeNet model has been properly trained on MNIST dataset.

### Deploying and Testing Adversarial Patches

Now we will create and apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.
We will also apply the patches over the training set for the adversarial training defense evaluation.

#### Patch Generation

The following job will generate the adversarial patches. 
Feel free to adjust the input parameters to see how they impact the effectiveness of the patch attack.

In [ ]:
# Create Patches
response_le_net_patches = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model_version=none",
            "-P num_patch_gen_samples=40",
            "-P num_patch=3",
            "-P patch_target=5",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)


print("Patch attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_le_net_patches)
print("")

# Wait for Patch attack to finish.
response_le_net_patches = wait_until_finished(response_le_net_patches)

#### Patch Deployment

The following jobs will deploy the patches over the training and test sets.

In [ ]:
# Deploy Patch attack on training set.

response_deploy_le_net_patches_mnist_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_patches['mlflowRunId']}",
            f"-P model_version=none",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_patches["jobId"],
)

print("Patch deployment (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_training)
print("")

# Get the run ID of the test set.
response_deploy_le_net_patches_mnist_training = get_run_id(response_deploy_le_net_patches_mnist_training)

In [ ]:
# Deploy Patch attack on test set.
response_deploy_le_net_patches_mnist_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_patches['mlflowRunId']}",
            f"-P model_version=none",
            "-P patch_deployment_method=corrupt",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_patches["jobId"],
)

print("Patch deployment (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_testing)
print("")

# Get the run ID of the training set.
response_deploy_le_net_patches_mnist_testing = get_run_id(response_deploy_le_net_patches_mnist_testing)

### Patch Attack Evaluation: Baseline MNIST Model

Now we will run an inference step to check the patch-attacked dataset with our MNIST-trained model.

In [ ]:
# Check patched dataset results
response_le_net_infer_le_net_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P model_version=none",
            "-P batch_size=512",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)

print("Patch evaluation (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_testing)
print("")

In [ ]:
# Wait for the job to finish
response_le_net_infer_le_net_patch = wait_until_finished(response_le_net_infer_le_net_patch)

# Check on the patch evaluation results
results = get_mlflow_results(response_le_net_infer_le_net_patch)
print("Baseline model results on adversarially patched dataset: ")
pprint.pprint(results.data.metrics)


We can see that the adversarial patch attack causes a noticeable decrease in the model's accuracy scores.

We will now test various defenses against the patch attacked images.

## Defenses: Image Preprocessing and Adversarial Training

The next part of the adversarial patch demo focuses on investigating effective defenses against the attack.

### Preprocessing Defenses: Spatial Smoothing, JPEG Compression, Gaussian Augmentation

Here we will investigate three preprocessing defenses that can be applied over the images before inference.

#### Spatial Smoothing Defense:

Here, we can adjust the `spatial_smoothing_window_size` parameter to increase or decrease the sliding window of the smoothing defense.
Larger values will create more noticeable distortions but can also help mask any adversarial perturbations.

In [ ]:
response_le_net_spatial_smoothing_test_set = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="spatial_smoothing",
    entry_point_kwargs=" ".join(
        [
            "-P batch_size=20",
            "-P spatial_smoothing_window_size=2",
            f"-P dataset_run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)

print("Spatial smoothing defense job submitted")
print("")
pprint.pprint(response_le_net_spatial_smoothing_test_set)
print("")

response_le_net_spatial_smoothing_test_set = get_run_id(response_le_net_spatial_smoothing_test_set)

In [ ]:
# Wait for defense to complete, then check baseline model on defended test set with adversarial patches.

response_evaluate_spatial_smoothing_images = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_spatial_smoothing_test_set['mlflowRunId']}",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=spatial_smoothing_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_spatial_smoothing_test_set["jobId"],
)

print("Inference job submitted")
print("")
pprint.pprint(response_evaluate_spatial_smoothing_images)
print("")

response_evaluate_spatial_smoothing_images = wait_until_finished(response_evaluate_spatial_smoothing_images)
results = get_mlflow_results(response_evaluate_spatial_smoothing_images)
print("Adversarial Patches with Spatial Smoothing Results: \n")
pprint.pprint(results.data.metrics)

It appears that spatial smoothing does not protect well against adversarial patches.

#### JPEG Compression Defense:

We can adjust the image compression quality by modifying the `jpeg_compression_quality` field. 
Enter any value between 1 (worst) to 95 (best).

In [ ]:
response_le_net_jpeg_compression_test_set = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="jpeg_compression",
    entry_point_kwargs=" ".join(
        [
            "-P batch_size=20",
            f"-P dataset_run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            "-P jpeg_compression_quality=30",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)

print("JPEG compression defense job submitted")
print("")
pprint.pprint(response_le_net_jpeg_compression_test_set)
print("")

response_le_net_jpeg_compression_test_set = get_run_id(response_le_net_jpeg_compression_test_set)

In [ ]:
# Wait for defense to complete, then check baseline model on defended test set with adversarial patches.

response_evaluate_jpeg_compression_images = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_jpeg_compression_test_set['mlflowRunId']}",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=jpeg_compression_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_jpeg_compression_test_set["jobId"],
)

print("Inference job submitted")
print("")
pprint.pprint(response_evaluate_jpeg_compression_images)
print("")

response_evaluate_jpeg_compression_images = wait_until_finished(response_evaluate_jpeg_compression_images)
results = get_mlflow_results(response_evaluate_jpeg_compression_images)
print("Adversarial Patches with JPEG Compression Results: \n")
pprint.pprint(results.data.metrics)


#### Gaussian Augmentation Defense:

We can adjust the amount of noise created by the defense by adjusting the `gaussian_augmentation_sigma` parameter. 
Please enter any positive value for sigma.

In [ ]:
response_le_net_gaussian_augmentation_test_set = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gaussian_augmentation",
    entry_point_kwargs=" ".join(
        [
            "-P batch_size=20",
            f"-P dataset_run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            "-P gaussian_augmentation_sigma=0.3",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)


print("Gaussian Augmentation defense job submitted")
print("")
pprint.pprint(response_le_net_gaussian_augmentation_test_set)
print("")

response_le_net_gaussian_augmentation_test_set = get_run_id(response_le_net_gaussian_augmentation_test_set)

In [ ]:
# Wait for defense to complete, then check baseline model on defended test set with adversarial patches.

response_evaluate_gaussian_augmentation_images = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_gaussian_augmentation_test_set['mlflowRunId']}",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=gaussian_augmentation_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_gaussian_augmentation_test_set["jobId"],
)

print("Inference job submitted")
print("")
pprint.pprint(response_evaluate_gaussian_augmentation_images)
print("")

response_evaluate_gaussian_augmentation_images = wait_until_finished(response_evaluate_gaussian_augmentation_images)
results = get_mlflow_results(response_evaluate_gaussian_augmentation_images)
print("Adversarial Patches with Gaussian Augmentation Results: \n")
pprint.pprint(results.data.metrics)


It appears that of the three preprocessing defenses, gaussian augmentation might actually improve patch attach effectiveness.

Since preprocessing defenses appear to be ineffective at stopping adversarial patches, let's try another approach.

## Adversarial Training Defense:

Finally, we will train a new copy of the LeNet model on training set that contains adversarial patches.
In doing so, the model learns to ignore the adversarial patches.

In [ ]:
# Finally, train and retest patched dataset.

response_deploy_le_net_patches_mnist_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train_on_Mnist_patched",
    entry_point_kwargs=" ".join(
        [
            f"-P dataset_run_id_testing={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P dataset_run_id_training={response_deploy_le_net_patches_mnist_training['mlflowRunId']}",
            "-P batch_size=256",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_training["jobId"],
)

print("Patch adversarial training (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_adv_training)
print("")

response_deploy_le_net_patches_mnist_adv_training = get_run_id(response_deploy_le_net_patches_mnist_adv_training)

In [ ]:
response_evaluate_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P model_version=none",
            "-P batch_size=20",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_adv_training["jobId"],
)

print("Patch evaluation job submitted")
print("")
pprint.pprint(response_evaluate_adv_training)
print("")

response_evaluate_adv_training = wait_until_finished(response_evaluate_adv_training)
results = get_mlflow_results(response_evaluate_adv_training)
print("Adversarial Training Results:")

**We can see that for adversarial patches, adversarial training appears to be a good option.**

**Some other interesting approaches that users may want to investigate by modfifying this demo are:**

-   Generating a separate set of patches for training and test data. 
    -   To do so, add a secondary patch generation job and link the run_id of this new job to one of the existing test/training patch deployment jobs.
-   Experimenting with alternate adversarially trained models. 
    -   If both the MNIST patch demo and MNIST FGM demos are run together, then users can add `model={name_of_FGM}` parameter to the `entry_point_kwargs` section in the example.
    -   Doing so can let users quickly check which adversarial training offers robust protection against multiple types of attacks. 
     
**Please consult the README documentation for more information regarding available entrypoints and attack/defense parameters.**